In [1]:
%pip install geopandas pandas pyshp geojson


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import geopandas as gpd
import geojson
import json

In [3]:
df = pd.read_csv('CO2 emission by countries.csv')
df.head()

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km�
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km�
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km�
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km�
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km�


In [4]:
df = df.loc[:, ['Country', 'Year', 'CO2 emission (Tons)']]
df.head()

,Country,Year,CO2 emission (Tons)
0,Afghanistan,1750,0.0
1,Afghanistan,1751,0.0
2,Afghanistan,1752,0.0
3,Afghanistan,1753,0.0
4,Afghanistan,1754,0.0


In [5]:
total_world_emissions = df.groupby('Year')['CO2 emission (Tons)'].sum().reset_index()
total_world_emissions = total_world_emissions.rename(columns={'CO2 emission (Tons)': 'Total World Emission (Tons)'})
total_world_emissions.head()

,Year,Total World Emission (Tons)
0,1750,9350528.0
1,1751,18701056.0
2,1752,28055248.0
3,1753,37409440.0
4,1754,46767296.0


In [6]:
df = df.merge(total_world_emissions, on='Year')
# # Rename the 'CO2 emission (Tons)' column from df2
# df = df.rename(columns={'Total World Emission (Tons)_y': 'Total World Emission (Tons)'})
# # Drop the duplicate 'CO2 emission (Tons)' column
# df = df.drop(columns=['Total World Emission (Tons)_x'])
# Display the final DataFrame with formatting
pd.options.display.float_format = '{:.1f}'.format
df.head()

,Country,Year,CO2 emission (Tons),Total World Emission (Tons)
0,Afghanistan,1750,0.0,9350528.0
1,Albania,1750,0.0,9350528.0
2,Algeria,1750,0.0,9350528.0
3,Andorra,1750,0.0,9350528.0
4,Angola,1750,0.0,9350528.0


In [7]:
output_folder = '../D3js-Visualization/data/'
output_file = output_folder + 'emissions_processed.csv'

df.to_csv(output_file, index=False)

In [8]:
# Download the Natural Earth dataset for countries with geometries
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

output_file = 'world_data.geojson'

world.to_file(output_file, driver='GeoJSON')


/var/folders/wp/wvs3l1cx7qj62cdcp32j1kpc0000gn/T/ipykernel_54117/1607004158.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [9]:
# path_processed = "../D3js-Visualization/data/emissions_processed.csv"
path_geojson = "world_data.geojson"

In [10]:
df

,Country,Year,CO2 emission (Tons),Total World Emission (Tons)
0,Afghanistan,1750,0.0,9350528.0
1,Albania,1750,0.0,9350528.0
2,Algeria,1750,0.0,9350528.0
3,Andorra,1750,0.0,9350528.0
4,Angola,1750,0.0,9350528.0
...,...,...,...,...
59615,Vietnam,2020,3977272058.0,1654087724978.0
59616,Wallis and Futuna,2020,507247.0,1654087724978.0
59617,Yemen,2020,624654156.0,1654087724978.0
59618,Zambia,2020,247987833.0,1654087724978.0


In [11]:
import json
with open(path_geojson, 'r') as geojson_file:
    geojson_data = json.load(geojson_file)


# # Print the loaded GeoJSON data
# print(json.dumps(geojson_data))  # This will print the data with indentation for better readability


In [12]:
countries_csv = []
countries_geojson = []
for feature in geojson_data['features']:
    countries_geojson.append(feature['properties']['name'])

for index, row in df.iterrows():
    country = row['Country']
    if country not in countries_csv:
        countries_csv.append(country)


result= list(set(countries_geojson) - set(countries_csv))
result

['Dem. Rep. Congo',
 'Timor-Leste',
 'Eq. Guinea',
 'N. Cyprus',
 'Fr. S. Antarctic Lands',
 'Dominican Rep.',
 'Falkland Is.',
 'eSwatini',
 "Côte d'Ivoire",
 'Somaliland',
 'W. Sahara',
 'Bosnia and Herz.',
 'United States of America',
 'S. Sudan',
 'Solomon Is.',
 'Central African Rep.']

In [13]:
# Create the dictionary
emission_dict = {}
custom_dict = {
    'Timor': 'Timor-Leste',
    'United States': 'United States of America',
    'Eswatini': 'eSwatini',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Solomon Islands': 'Solomon Is.',
    'Central African Republic': 'Central African Rep.',
    'South Sudan': 'S. Sudan',
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Equatorial Guinea": "Eq. Guinea",
    "Dominican Republic": "Dominican Rep.",
    "Democratic Republic of Congo": "Dem. Rep. Congo",
    "Antartica": "Fr. S. Antarctic Lands"
}

for index, row in df.iterrows():
    country = row['Country']
    year = int(row['Year'])
    # print(country,year)
    emission = row['CO2 emission (Tons)']
    if country in custom_dict:
        country = custom_dict[country]
    if country not in emission_dict:
        emission_dict[country] = {}
    emission_dict[country][str(year)] = emission

# Print the resulting dictionary
# print(emission_dict,indent=2)
# print(json.dumps(emission_dict))


In [14]:
# Assuming geojson_data is the loaded GeoJSON data (as a Python dictionary) and emission_dict is the dictionary you created

# Iterate through the features in the GeoJSON data
custom_emission_dict={}
for i in range(1750,2021):
    custom_emission_dict[str(i)] = 0
for feature in geojson_data['features']:
    country_name = feature['properties']['name']

    if country_name in emission_dict:
        feature['properties']['emission_data'] = emission_dict[country_name]
    else:
        feature['properties']['emission_data'] = custom_emission_dict
# Save the updated GeoJSON data to a new file
with open('../D3js-Visualization/data/world_data.geojson', 'w') as outfile:
    json.dump(geojson_data, outfile)
